## GA-output

Nb to test the output files and params of  the GASS

In [1]:
using PyCall
using DataFrames

rootdir = "/home/stephane/Science/ALMA/ArrayConfig/GASS"
#rootdir = "/home/stephane/alma/ArrayConfig/GASS"

push!(LOAD_PATH,"$rootdir/master/src")
using GASS
using Printf

import PyPlot
import Random
import JLD

np= pyimport("numpy")


## directory
datadir = "$rootdir/master/data"
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/test"

cd(wdir)

┌ Info: Recompiling stale cache file /home/stephane/.julia/compiled/v1.1/GASS.ji for GASS [top-level]
└ @ Base loading.jl:1184


In [2]:
macro main(inpfile)
    cfg = read_cfg(inpfile , verbose=true)
    
    res= gass_optimization(cfg)
    
    return(res)
end

res= @main("../master/data/GA_Inputs_O-3.txt.julia")

## Input Parameters for GASS 
### Configuration file: ../master/data/O-3.cfg 
### Obs. Latitude: -23.026 
### Source Declination: -50.0 
### HA: 0.0 
### Antenna number: 50 
### Subarray number: 4 
##
## Subarray Parameters
### Pads per subarray: [30, 10, 7, 3]
### Name: ["Sub_1", "Sub_2", "Sub_3", "Sub_4"]
### AR: [2.3, 3.0, 3.5, 3.5]
### MRS: [20.0, 20.0, 10.0, 10.0]
### elongation: [1.2, 1.3, 2.0, 3.0]
### sidelobe: [10.0, 20.0, 60.0, 60.0]
##
## GA parameters
### Iterations: 5 
### Population size: 10 
### Mutation rate: 0.050 
### Tournament size: 5 
### Elitism: 5 
##
## Weights
### Subarray weights: [0.25, 0.25, 0.25, 0.25]
### AR weights: [0.25, 0.25, 0.25, 0.25]
### MRS weights: [0.25, 0.25, 0.25, 0.25]
### elongation weights: [0.25, 0.25, 0.25, 0.25]
### sidelobe weights: [0.25, 0.25, 0.25, 0.25]
##

##
## Creating the population...
1 1 2.797680981919089
Any[synthbeam(2.48039, 2.57147, 2.52552, 1.03672, 7.06643), 5.93794]
1 2 0.2910530263579978
Any[synthbeam(3.25001, 2.34616,

10 1 3.1847639754968755
10 2 0.1224366584482941
10 3 -16.351192435747656
10 4 -11.915418082613066

## Iteration: 2 
(1, 2)
[44, 45, 12, 37, 7, 8, 17, 19, 42, 26, 14, 30, 29, 39, 20, 13, 4, 48, 46, 47, 21, 18, 3, 41, 24, 16, 33, 11, 28, 1, 25, 22, 36, 31, 49, 9, 50, 10, 27, 40, 23, 34, 43, 35, 38, 5, 6, 2, 15, 32]
(1, 7)
[37, 26, 13, 38, 22, 15, 17, 20, 8, 12, 39, 4, 16, 19, 36, 42, 14, 21, 2, 18, 3, 28, 5, 50, 41, 10, 33, 9, 1, 48, 43, 23, 31, 11, 46, 6, 25, 24, 44, 47, 35, 34, 27, 40, 32, 45, 49, 7, 29, 30]
(7, 10)
[44, 48, 46, 47, 8, 21, 13, 18, 3, 41, 19, 16, 4, 33, 5, 28, 1, 25, 39, 24, 36, 42, 37, 30, 31, 17, 12, 9, 50, 10, 14, 20, 27, 23, 34, 43, 35, 11, 6, 2, 22, 38, 15, 40, 32, 45, 49, 7, 29, 26]
(1, 10)
[16, 36, 12, 37, 7, 8, 17, 19, 42, 26, 14, 30, 29, 39, 20, 13, 4, 22, 2, 18, 3, 28, 5, 50, 41, 10, 33, 9, 1, 48, 43, 23, 31, 11, 46, 6, 25, 24, 44, 47, 35, 34, 27, 40, 32, 45, 49, 38, 15, 21]
(2, 1)
[16, 36, 12, 37, 7, 8, 17, 19, 42, 26, 30, 29, 39, 13, 4, 22, 2, 18, 3, 28, 5, 

1 1 -32.89054156963181
1 2 0.9129145745667513
1 3 -14.519344094478052
1 4 -8.298763323768997
2 1 1.9850114970816337
2 2 0.8009484319237132
2 3 -15.750286239683385
2 4 -13.210794658268307
3 1 2.129728411834063
3 2 1.0300645879091723
3 3 -15.750286239683385
3 4 -13.210794658268307
4 1 2.313600173022122
4 2 0.9788655001641962
4 3 -15.750286239683385
4 4 -13.210794658268307
5 1 1.3188420986596336
5 2 0.8009484319237132
5 3 -15.750286239683385
5 4 -11.833138551138704
fitness of crossover..
[37, 26, 18, 7, 38, 22, 15, 2, 17, 20, 8, 12, 39, 4, 16, 44, 3, 42, 14, 27, 10, 40, 5, 50, 41, 36, 33, 9, 1, 48, 43, 23, 31, 11, 46, 6, 25, 24, 19, 47, 35, 34, 21, 28, 32, 45, 49, 13, 29, 30]
Array{Int64,1}[[37, 26, 18, 7, 38, 22, 15, 2, 17, 20, 8, 12, 39, 4, 16, 44, 3, 42, 14, 27, 10, 40, 5, 50, 41, 36, 33, 9, 1, 48], [43, 23, 31, 11, 46, 6, 25, 24, 19, 47], [35, 34, 21, 28, 32, 45, 49], [13, 29, 30]]
6 1 2.318145908992803
6 2 1.052728071677659
6 3 -17.248201711463075
6 4 -9.218717586231564
[31, 26, 13, 

5-element Array{Any,1}:
 population(1, Array{Int64,1}[[16, 36, 12, 37, 7, 8, 17, 19, 42, 26  …  3, 28, 5, 50, 41, 10, 33, 9, 1, 48] [43, 23, 31, 11, 46, 6, 25, 24, 44, 47] [35, 34, 27, 40, 32, 45, 49] [38, 15, 21]; [44, 45, 48, 46, 47, 8, 21, 13, 18, 3  …  39, 22, 36, 42, 37, 30, 31, 49, 7, 17] [29, 12, 9, 50, 10, 14, 20, 27, 40, 23] [34, 43, 35, 38, 5, 6, 2] [15, 32, 26]; … ; [48, 47, 21, 13, 18, 33, 11, 22, 36, 42  …  1, 32, 25, 28, 3, 20, 8, 12, 39, 41] [31, 4, 44, 16, 43, 19, 49, 7, 29, 9] [10, 14, 27, 40, 23, 34, 35] [38, 5, 26]; [44, 48, 46, 47, 8, 21, 13, 18, 3, 41  …  24, 36, 42, 37, 30, 31, 7, 17, 29, 12] [9, 50, 10, 14, 20, 27, 40, 23, 34, 43] [35, 11, 6, 2, 32, 22, 49] [38, 15, 26]], [1.31884 0.800948 -15.7503 -11.8331; 2.60296 -1.34766 -12.0641 -11.8572; … ; 1.95713 -0.599326 -11.8595 13.3884; 3.18476 0.122437 -16.3512 -11.9154], [6.36591, 5.66649, 5.57935, 3.67276, 1.75766, 5.23158, 6.54378, 6.14114, -0.721685, 6.23985], Dict{String,Float64}[Dict("mrs"=>12.418,"ar"=>2.9089

In [3]:
println(res[1].age)
println(res[1].fitness)
println(res[5].score)

JLD.save("test.jld", "evol", res)

1
[1.31884 0.800948 -15.7503 -11.8331; 2.60296 -1.34766 -12.0641 -11.8572; 2.79768 0.291053 -13.1501 -12.2561; -1.66402 -2.34448 -18.3745 7.69198; -2.56507 -0.294769 -11.3079 7.13704; 2.77727 0.99153 -11.2597 -13.4354; 1.98501 0.800948 -15.7503 -13.2108; 2.45001 1.45006 -13.1501 -15.3145; 1.95713 -0.599326 -11.8595 13.3884; 3.18476 0.122437 -16.3512 -11.9154]
[13.6989, 7.53732, 7.48221, 6.54378, 6.45032, 12.9911, 7.23941, -0.423707, 5.34091, 5.51654]


In [4]:
a = JLD.load("test.jld")
a["evol"][5].subarr[5,2]

10-element Array{Int64,1}:
 43
 23
 37
 11
 46
  6
 10
 24
 44
 47

In [11]:
### save array in CASA format..
###
function _save_CASAarr(fileprefix::String, cfg::cfg, bestsub)
    println("Saving results")
    println(cfg.arr)
    println(cfg.sub)
    #println(p.subarr)
    #println(p.score)
    #println(p.fitness)
    
    headerCasa= "# observatory=ALMA\n# coordsys=LOC (local tangent plane)\n# x y z diam pad#\n"
    
    for i in 1:length(bestsub)
        arrStr= string(headerCasa)
        subname= cfg.sub.Subarray_Name[i]
        # println(cfg.arr.X[bestsub[i]])
        println(length(bestsub[i]))
        for j in 1:length(bestsub[i])
            # println(cfg.arr.X[bestsub[i][j]])
            arrStr *= @sprintf("%f %f %f %2.1f %s \n",cfg.arr.X[bestsub[i][j]], cfg.arr.Y[bestsub[i][j]] , 
                cfg.arr.Z[bestsub[i][j]] , cfg.arr.diam[bestsub[i][j]] , cfg.arr.name[bestsub[i][j]])
        end
        println(arrStr)
        filename= fileprefix*subname*".cfg"
        println(fileprefix)
        println(subname)
        println(filename)
        f= open(filename, "w")
        print(f,arrStr)
        close(f)
    end
end

## sort population wrt. score
##
function _sort_population(p::population)
    println("Sorting..")
    s= sortperm(p.score, rev=true)
    println(s)
    psorted= population(p.age, p.subarr[s,:], p.fitness[s,:], p.score[s], p.param[s,:])
    println(psorted.fitness)
    return(psorted)
end

_sort_population (generic function with 1 method)

In [12]:
inpfile= "../master/data/GA_Inputs_O-3.txt.julia"
a= read_cfg(inpfile , verbose=true)
println(typeof(res[1]))

println

psort= _sort_population(res[5])
## psort.subarr[1,:] is the first best solution...
#println(psort.subarr[1,:])
_save_CASAarr("toto-",a, psort.subarr[1,:])


## Input Parameters for GASS 
### Configuration file: ../master/data/O-3.cfg 
### Obs. Latitude: -23.026 
### Source Declination: -50.0 
### HA: 0.0 
### Antenna number: 50 
### Subarray number: 4 
##
## Subarray Parameters
### Pads per subarray: [30, 10, 7, 3]
### Name: ["Sub_1", "Sub_2", "Sub_3", "Sub_4"]
### AR: [2.3, 3.0, 3.5, 3.5]
### MRS: [20.0, 20.0, 10.0, 10.0]
### elongation: [1.2, 1.3, 2.0, 3.0]
### sidelobe: [10.0, 20.0, 60.0, 60.0]
##
## GA parameters
### Iterations: 5 
### Population size: 10 
### Mutation rate: 0.050 
### Tournament size: 5 
### Elitism: 5 
##
## Weights
### Subarray weights: [0.25, 0.25, 0.25, 0.25]
### AR weights: [0.25, 0.25, 0.25, 0.25]
### MRS weights: [0.25, 0.25, 0.25, 0.25]
### elongation weights: [0.25, 0.25, 0.25, 0.25]
### sidelobe weights: [0.25, 0.25, 0.25, 0.25]
##
population
Sorting..
[1, 6, 2, 3, 7, 4, 5, 10, 9, 8]
[-32.8905 0.912915 -14.5193 -8.29876; -32.7674 0.682305 -11.5805 -8.29876; 2.0829 -3.78805 -15.2334 -13.2108; 3.13549 0.58519 